In [ ]:
%matplotlib notebook
#%matplotlib inline

import matplotlib.pyplot as plt
import rasterio
import rasterio.warp
import numpy as np
import math
from scipy import ndimage as ndi

import pyproj
import xarray as xr
import fiona
import shapely
import pydash
from osgeo import osr
from numpy import pi, r_, c_, vstack, hstack
import sys

from kk.dggs import DGGS
from kk.tools import polygon_path, spread_num_samples, gen_pts_from_distribution

In [ ]:
def get_affine(fname):
    with rasterio.open(fname) as f:
        return f.affine

def get_poly_lines():
    with fiona.open('/Users/kirill/wk/dggs/qgis/data/cstauscd_l.shp') as f:
        return [np.r_[ pydash.get(p, 'geometry.coordinates') ].T 
                for p in f if pydash.get(p, 'properties.FEAT_CODE') == 'tile_edge']

In [ ]:
filename = '/Users/kirill/wk/dggs/abs_data/apg16e1_1_1_0.tif'

In [ ]:
A = get_affine(filename)
ds = xr.open_rasterio(filename)
ds.attrs['affine'] = A
ds.values[ds.values<0] = np.nan

src = ds.isel(band=0)
ds.sum(), src.sum()

In [ ]:
plt.figure()
src.plot.imshow()
plt.axis('equal')

In [ ]:
dg = DGGS()

In [ ]:
tr, (maxW, maxH) = dg.pixel_coord_transform('S30000000', dst_proj=ds.crs)

xx, yy = tr(*polygon_path(np.arange(3**7), np.arange(3**7)))

plt.figure()
plt.plot(xx,yy, 'y.')
src.plot.imshow()

xx_, yy_ = polygon_path(ds.x, ds.y)
plt.plot(xx_, yy_, 'b.')
plt.axis('tight')
plt.axis('equal')




In [ ]:
np.abs(np.diff(yy)).mean()

In [ ]:
wrp = dg.mk_warper('R60000000', 3*3**7, 3**7, src_proj=ds.crs)

In [ ]:
im = wrp(ds.values[0], ds.affine)

In [ ]:
plt.figure()
plt.imshow(im)

In [ ]:
np.sum(im[np.isfinite(im)])

In [ ]:
np.arange(9).reshape(3,3)